Imports
============

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from dvn.dvn_fcn import DeepVesselNetFCN
from dvn.solver import Solver
from dvn.data_utils import SyntheticData

torch.set_default_tensor_type('torch.FloatTensor')
# set up default cuda device
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Firstly we load the data
============


In [2]:
patch_size=64

train_synthetic = SyntheticData(root_path="./data/train/", patch_size=patch_size)
val_synthetic = SyntheticData(root_path="./data/val/", patch_size=patch_size)

print("DONE")

DONE


Visualize part of the data
============

In [ ]:
print("Train size: %i" % len(train_synthetic))
print("Validation size: %i" % len(val_synthetic))
print("Img size: ", train_synthetic[0][0].size())
print("Segmentation size: ", train_synthetic[0][1].size())

Train size: 110
Validation size: 26


Design network 
============

In [ ]:
train_loader = torch.utils.data.DataLoader(train_synthetic[0:10], batch_size=2, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_synthetic[0:5], batch_size=1, shuffle=False, num_workers=0)

model = DeepVesselNetFCN()
model.to(device)

Train network
============

In [ ]:
solver = Solver(optim_args={"lr": 0.001, "weight_decay": 0.05})
solver.train(model, train_loader, val_loader, log_nth=5, num_epochs=5)


Test network
============

Visualization of network outputs
============
